In [ ]:
import gdal
from gdalconst import *
import pandas as pd
import numpy
from skimage import io as sk_io
import matplotlib.pyplot as plt
import skimage as sk

In [ ]:
#images without railways
data_not = pd.read_csv('/content/drive/My Drive/Aerocosmos/Codes/points2000.csv', sep =',')
xValues = data_not['X'].values
yValues = data_not['Y'].values
data_not.head()

In [ ]:
#images with railways
data = pd.read_csv('/content/drive/My Drive/Aerocosmos/Codes/points8000.csv', sep =',')
xValues = data['x'].values
yValues = data['y'].values

In [ ]:
#converting points from one coordinate system to another
!pip install pyproj
from pyproj import Proj, transform

#inProj = Proj(init='epsg:4326')
#outProj = Proj(init='epsg:32637')
inProj = Proj(init='epsg:32637')
outProj = Proj(init='epsg:3857')
x1,y1 = xValues[100], yValues[100]
x2,y2 = transform(inProj,outProj,x1,y1)
print(x2,y2)

#xValues, yValues = [], []

#for i in range(xVal.size):
#  x,y = transform(inProj,outProj,xVal[i],yVal[i])
#  xValues.append(x)
#  yValues.append(y)


In [ ]:
gdal.AllRegister()
ds = gdal.Open('/content/drive/My Drive/Aerocosmos/MUL_ACOMP.tif', GA_ReadOnly)
if ds is None:
  print('Could not open image')

In [ ]:
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount

transform = ds.GetGeoTransform()

x0rigin = transform[0]
y0rigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]

In [ ]:
#finding the pixel for the corresponding pair of coordinates

#!pip install pyproj
'''import pyproj
from pyproj import Proj, transform
inProj = Proj(init='epsg:32637')
outProj = Proj(init='epsg:3857')
'''
X_pixels = []
Y_pixels = []

for i in range(xValues.size):
#for i in range(3):
  x = xValues[i]
  y = yValues[i]
  #x,y = pyproj.transform(inProj,outProj,xValues[i],yValues[i])

  xOffset = int((x-x0rigin) / pixelWidth)
  yOffset = int((y-y0rigin) / pixelHeight)
  
  X_pixels.append(xOffset)
  Y_pixels.append(yOffset)

In [ ]:
#crop process
size = (512, 512)

def check_x(x):
    if not (x<0 & x>cols):
        return False
    else:
      return True

def check_y(y):
    return ((y<0)  and (y>rows))

def cut(x,y, size):
    x_left = x - int(size[0]/2)
    x_right = x + int(size[0]/2)
    y_down = y - int(size[1]/2)
    y_up = y + int(size[1]/2) 

    if size[0] % 2 != 0 :
        x_right+=1

    if size[1] % 2 != 0 :
        y_up+=1

    if (check_x(x_left) & check_x(x_right) & check_y(y_down) & check_y(y_up)):
        return 0

    crop = ds.ReadAsArray(x_left, y_down, size[0], size[1])
    
    x0 = x_left * pixelWidth + x0rigin
    y0 = y_down * pixelHeight + y0rigin

    return crop, x0, y0

In [ ]:
#save images
k = 10001
#for i in range(3):
for i in range(len(X_pixels)):
    cropped, x, y = cut(X_pixels[i], Y_pixels[i], size)
      s = '/content/drive/My Drive/Aerocosmos/my_source/data/mul/mul_tif/' + str(k)[1:]+'.tif'
      k+=1
      format = "GTiff"
      driver = gdal.GetDriverByName( format )
      #dt = gdal.GDT_Byte
      dt = gdal.GDT_UInt16
      projection = ds.GetProjection()
      transform_img = list(ds.GetGeoTransform())
      transform_img[0] = x
      transform_img[3] = y
      outData = driver.Create(s, size[0], size[1], bands, dt )
      outData.SetProjection( projection )
      outData.SetGeoTransform( tuple(transform_img))
      #outData.GetRasterBand(1).WriteArray( cropped )
      for i in range( bands ):
          outData.GetRasterBand( i + 1 ).WriteArray( cropped[ i ] )
      outData = None
    else:
      kk.append(str(k)[1:])
      k+=1

In [ ]:
#to check how many files in a directory
import os

def fcount(path):
       """ Counts the number of files in a directory """
       count = 0
       for f in os.listdir(path):
           if os.path.isfile(os.path.join(path, f)):
               count += 1

       return count


# The following line prints the number of files in the current directory:
path = "/content/drive/My Drive/Aerocosmos/data/mul512/"
print(fcount(path))